In [14]:
import pandas as pd
import numpy as np
import datetime
import pickle
from tqdm import tqdm
import csv

In [15]:
from os import listdir
from os.path import isfile, isdir, join

# 指定要列出所有檔案的目錄
mypath = './dien_dart'
# 取得所有檔案與子目錄名稱
files = listdir(mypath)
csv_list = []
# 以迴圈處理
for f in files:
  # 產生檔案的絕對路徑
  fullpath = join(mypath, f)
  # 判斷 fullpath 是檔案還是目錄
  if isfile(fullpath):
#     print("檔案：", f)
    if f[-4:]=='.csv':
        csv_list.append(f[17:-4])
csv_list.sort()

In [16]:
csv_list

['20190315',
 '20190316',
 '20190317',
 '20190318',
 '20190319',
 '20190320',
 '20190321',
 '20190322',
 '20190323',
 '20190324',
 '20190325',
 '20190326',
 '20190327',
 '20190328',
 '20190329',
 '20190330',
 '20190331',
 '20190401',
 '20190402',
 '20190403',
 '20190404',
 '20190405',
 '20190406',
 '20190407',
 '20190408',
 '20190409',
 '20190410',
 '20190411',
 '20190412',
 '20190413',
 '20190414',
 '20190415',
 '20190416',
 '20190417',
 '20190418',
 '20190419',
 '20190420',
 '20190421',
 '20190422',
 '20190423',
 '20190424',
 '20190425',
 '20190426',
 '20190427',
 '20190428',
 '20190429',
 '20190430',
 '20190501',
 '20190502',
 '20190503',
 '20190504',
 '20190505',
 '20190506',
 '20190507',
 '20190508',
 '20190509',
 '20190510',
 '20190511',
 '20190512',
 '20190513',
 '20190514',
 '20190515',
 '20190516',
 '20190517',
 '20190518',
 '20190519',
 '20190520',
 '20190521',
 '20190522',
 '20190523',
 '20190524',
 '20190525',
 '20190526',
 '20190527',
 '20190528',
 '20190529',
 '20190530',

In [17]:
with open('../CTR_v4/item_data/dict_knn_table.pkl', 'rb') as file:
    dict_knn_table = pickle.load(file)

In [18]:
def knn_ranking(r):
    if r.target_iid in dict_knn_table:
        ids = dict_knn_table[r.target_iid]
#             ids, cos = zip(*lt)
        try:
            rank = ids.index(r.next_iid) + 1
        except:
            rank = np.nan
    else:
        rank = np.nan
    return rank

In [19]:
NN_tables = []
for part in range(1, 5):
    with open('../CTR_v4/dict_NN_table/NN_table_'+ str(part) +'.pkl', 'rb') as file:
        NN_tables.append(pickle.load(file))

In [20]:
def NN_ranking(r):
    for part in range(4):
        if r.target_iid in NN_tables[part]:
            lt = NN_tables[part][r.target_iid]
            ids, score = zip(*lt)
            try:
                rank = ids.index(r.next_iid) + 1
            except:
                rank = np.nan
            break
    else:
        rank = np.nan
    return rank

In [21]:
lt_df = []
for date in tqdm(csv_list):
    df = pd.read_csv('./dien_dart/' + 'change_dien_rank_' + date + '.csv').drop('Unnamed: 0', axis=1)    
    df.rename(columns = {'dien_rank':'DIEN_rank'}, inplace=True)
    df.sort_values(by=['uid'], inplace=True, ignore_index=True)
        
    dff = pd.read_csv('./din_dart/' + 'index_to_name_test_' + date + '_result' + '.csv')
    dff.rename(columns = {'din_rank':'DIN_rank'}, inplace=True)
    dff.sort_values(by=['uid'], inplace=True, ignore_index=True)
    dff.drop(columns=['hist_iid', 'last_iid', 'target_iid'], inplace=True)
    try:
        pd.testing.assert_series_equal(df['uid'], dff['uid'])
        df['DIN_rank'] = dff['DIN_rank']
    except Exception as e:
        print(date)
        print(e)
        df = pd.merge(df, dff, on=['uid'], how='left')
        df.fillna(dff['DIN_rank'].mean(), inplace=True)

    df.rename(columns = {'last_iid':'target_iid', 'target_iid':'next_iid'}, inplace=True)
    df['NN_rank'] = df.apply(NN_ranking, axis=1)
    df['knn_rank'] = df.apply(knn_ranking, axis=1)
    
    lt_df.append(df)

  0%|          | 0/108 [00:00<?, ?it/s]

20190315
Series are different

Series length are different
[left]:  6557, RangeIndex(start=0, stop=6557, step=1)
[right]: 6556, RangeIndex(start=0, stop=6556, step=1)


  1%|          | 1/108 [00:00<01:02,  1.70it/s]

20190316
Series are different

Series length are different
[left]:  5789, RangeIndex(start=0, stop=5789, step=1)
[right]: 5788, RangeIndex(start=0, stop=5788, step=1)


  3%|▎         | 3/108 [00:01<00:59,  1.75it/s]

...

 98%|█████████▊| 106/108 [01:07<00:00,  2.02it/s]

20190629
Series are different

Series length are different
[left]:  4769, RangeIndex(start=0, stop=4769, step=1)
[right]: 4768, RangeIndex(start=0, stop=4768, step=1)


 99%|█████████▉| 107/108 [01:08<00:00,  2.10it/s]

20190630
Series are different

Series length are different
[left]:  5251, RangeIndex(start=0, stop=5251, step=1)
[right]: 5250, RangeIndex(start=0, stop=5250, step=1)


100%|██████████| 108/108 [01:08<00:00,  1.57it/s]


In [22]:
with open('./lt_df_dart_108days.pkl', 'wb') as file:
    pickle.dump(lt_df, file)

In [23]:
lt_df[0]

,uid,hist_iid,target_iid,next_iid,DIEN_rank,DIN_rank,NN_rank,knn_rank
0,00071285-58af-4f79-bdcb-fe6be966464c,e33e0592b86611e99f6eacde48001122,e33e0592b86611e99f6eacde48001122,e32b67f4b86611e9bc33acde48001122,35,81.0,4,16
1,0016e69b-50e1-4aa7-90aa-9aec76b554ab,a8853924b86711e9aebcacde48001122/ba76834ab8671...,ba7a8f6cb86711e98024acde48001122,ba76d4b4b86711e98d41acde48001122,84,47.0,21,59
2,0021575a-bdc2-43bf-b6ef-e5414a1ec2d1,e328d2eeb86611e9a1deacde48001122,e328d2eeb86611e9a1deacde48001122,06f0a648b86711e9b66eacde48001122,5,42.0,8,5
3,0022550c-5c81-480d-b196-c3a370ce2cf6,00b66582b86811e98929acde48001122/06f35c26b8671...,00b66582b86811e98929acde48001122,06f35c26b86711e99f45acde48001122,85,16.0,5,1
4,0023d516-3dd0-4fc6-b10b-514152f6701c,e3252218b86611e99d55acde48001122/47183268b8671...,47183268b86711e9acd6acde48001122,471831fab86711e98ae4acde48001122,12,75.0,2,3
...,...,...,...,...,...,...,...,...
6552,ffbc79ac-65d9-4ef7-bc8f-a867edbfbc66,e34fe668b86611e99ad7acde48001122/e335ae10b8661...,e335ae10b86611e99ad4acde48001122,06f5211eb86711e98be4acde48001122,44,16.0,60,15
6553,ffc63cdf-6cd4-4c0b-bc7b-77ba79d1c3f1,7e6f5be2b86711e98b6facde48001122/7e98ebd8b8671...,072045a6b86711e99d06acde48001122,46d06168b86711e9bf47acde48001122,52,13.0,9,42
6554,ffde1bcb-3cf6-4f35-a66e-74889c5256d2,e378641cb86611e98abbacde48001122,e378641cb86611e98abbacde48001122,e379daaeb86611e9b229acde48001122,62,63.0,5,3
6555,ffeeed4a-45a7-4825-8dc8-d8f60a8e0067,5bce79b0b86711e9bd50acde48001122/5bbf8cb6b8671...,5bce0a8cb86711e9a52cacde48001122,06e11214b86711e9a47facde48001122,5,15.0,22,4


In [25]:
lt_df[100].describe()

,DIEN_rank,DIN_rank,NN_rank,knn_rank
count,6176.000000,6176.000000,6176.000000,6176.000000
mean,46.633258,43.973603,21.156412,21.156088
std,30.067202,27.177493,22.706631,24.473241
min,1.000000,1.000000,1.000000,1.000000
25%,17.000000,20.000000,4.000000,3.000000
50%,47.000000,42.000000,12.000000,11.000000
75%,73.000000,66.000000,31.000000,31.000000
max,99.000000,99.000000,99.000000,99.000000
